In [1]:
import struct

def density_file_reading(path):
    res = {}
    data_dict_lis = []
    moy_lis = []  #平均值, x, y, phase, e,r, z, dp/p
    moy2_lis = [] #平方值
    maxb_lis = [] #最大偏移
    minb_lis = [] #最小偏移

    maxR_lis = [] #最大光束尺寸或颗粒的最小值
    minR_lis = [] #最小尺寸的最大值

    rms_size_lis = [] #rms尺寸
    rms_size2_lis = [] #Squared beam rms size

    min_pos_moy_lis = []  #Min. if the beam average
    max_pos_moy_lis = []  #Maximum if the beam average

    rms_emit_lis = [] #rms emittances, xx’, yy’, zdp (m.rad)*/
    rms_emit2_lis = [] #Squared rms emittances, xx’, yy’, zdp

    lost_lis = []  #束损
    Milost_lis = [] #最小束损
    Malost_lis = [] #最大束损

    tab_lis = []
    stab_lis = []
    tabp_lis = []
    with open(path, "rb") as f:
        # 获取文件大小
        f.seek(0, 2)  # 移动到文件末尾
        longfichier = f.tell()
        f.seek(0, 0)  # 返回文件开头
        print(9)
        while True:
            # 读取固定大小的数据块
            data_dict = {}
            ver, year, vlong, Nrun, nelp, ib, Zg, Xouv, Youv, dXouv, dYouv, step = struct.unpack("<hhhiiffffffi", f.read(42))

            data_dict["ver"] = ver
            data_dict["year"] = year
            data_dict["vlong"] = vlong
            data_dict["Nrun步数"] = Nrun
            data_dict["nelp 元件"] = nelp
            data_dict["ib"] = ib
            data_dict["Zg 位置"] = Zg
            data_dict["Xouv 横向孔径"] = Xouv
            data_dict["Youv 纵向孔径"] = Youv
            data_dict["dXouv  水平孔径位移"] = dXouv
            data_dict["dYouv 纵向孔径位移"] = dYouv
            data_dict["step 步数"] = step
            data_dict_lis.append(data_dict)
            # print(data_dict)

            moy = struct.unpack("<fffffff", f.read(28))
            # print("平均值")
            # print(moy)
            moy_lis.append(moy)

            moy2 = struct.unpack("<fffffff", f.read(28))
            # print("平方值")
            # print(moy2)
            moy2_lis.append(moy2)

            maxb = struct.unpack("<fffffff", f.read(28))
            # print("最大尺寸")
            # print(maxb)
            maxb_lis.append(maxb)

            minb = struct.unpack("<fffffff", f.read(28))
            # print("最小尺寸")
            # print(minb)
            minb_lis.append(minb)

            if ver >= 11:
                #绝对相位，
                phaseF, phaseG = struct.unpack("<ff", f.read(8))
                # print("绝对参考相位")
                # print(phaseF, phaseG)

            if ver >= 10:
                #最大尺寸或者最小尺寸
                maxR = struct.unpack("<fffffff", f.read(28))
                minR = struct.unpack("<fffffff", f.read(28))
                # print("最大尺寸或者最小尺寸")
                # print(maxR)
                # print(minR)
                maxR_lis.append(maxR)
                minR_lis.append(minR)

            if ver >= 5:
                #rms尺寸
                rms_size = struct.unpack("<fffffff", f.read(28))
                rms_size2 = struct.unpack("<fffffff", f.read(28))
                # print("rms尺寸")
                # print(rms_size)
                # print(rms_size2)
                rms_size_lis.append(rms_size)
                rms_size2_lis.append(rms_size2)

            if ver >= 6:
                print("")
                min_pos_moy = struct.unpack("<fffffff", f.read(28))
                max_pos_moy = struct.unpack("<fffffff", f.read(28))
                # print(min_pos_moy)
                # print(max_pos_moy)
                min_pos_moy_lis.append(min_pos_moy)
                max_pos_moy_lis.append(max_pos_moy)

            if ver >= 7:
                rms_emit = struct.unpack("<fff", f.read(12))
                rms_emit2 = struct.unpack("<fff", f.read(12))
                # print("rms发射度")
                # print(rms_emit)
                # print(rms_emit2)
                rms_emit_lis.append(rms_emit)
                rms_emit2_lis.append(rms_emit2)

            if ver >= 8:
                Eouv, PhPouv, PhMouv = struct.unpack("<fff", f.read(12))
                # print("能量,  正向接收，负向接收")
                # print(Eouv, PhPouv, PhMouv)

            Np = struct.unpack("<q", f.read(8))[0]
            # print("粒子数", Np)
            if Np > 0:
                # 读取粒子损失和束流功率损失数据
                lost = []
                powlost = []
                for i in range(Nrun):
                    lost.append(struct.unpack("<q", f.read(8))[0])
                    powlost.append(struct.unpack("<f", f.read(4))[0])
                # print(lost)
                # print(powlost)
                lost_lis.append(lost)

                #损失的平方
                lost2 = struct.unpack("<q", f.read(8))[0]
                # print(lost2)
                #最小损失

                Milost = struct.unpack("<q", f.read(8))[0]
                # print(Milost)
                Milost_lis.append(Milost)

                #最大损失
                Malost = struct.unpack("<q", f.read(8))[0]
                # print(Malost)
                Malost_lis.append(Malost)

                powlost2 = struct.unpack("<d", f.read(8))[0]

                Mipowlost = struct.unpack("<f", f.read(4))[0]
                Mapowlost = struct.unpack("<f", f.read(4))[0]

                # 读取束流分布数据
                tab = [[] for _ in range(7)]
                stab = [[] for _ in range(7)]

                for j in range(7):
                    if vlong == 1:
                        tab[j] = struct.unpack(f"<{step}Q", f.read(step * 8))
                    else:
                        stab[j] = struct.unpack(f"<{step}I", f.read(step * 4))
                # print(tab[0])
                stab_lis.append(stab)
                if ib > 0:
                    tabp = [[] for _ in range(3)]

                    for j in range(3):
                        tabp[j] = struct.unpack(f"<{step}f", f.read(step * 4))
                tabp_lis.append(tabp)
            # 跳出循环条件
            if f.tell() + 16 >= longfichier:
                break
    res["data_dict_lis"] = data_dict_lis
    res["moy_lis"] = moy_lis
    res["moy2_lis"] = moy2_lis
    res["maxb_lis"] = maxb_lis
    res["minb_lis"] = minb_lis
    res["maxR_lis"] = maxR_lis
    res["minR_lis"] = minR_lis
    res["rms_size_lis"] = rms_size_lis
    res["rms_size2_lis"] = rms_size2_lis
    res["min_pos_moy_lis"] = min_pos_moy_lis
    res["max_pos_moy_lis"] = max_pos_moy_lis
    res["rms_emit_lis"] = rms_emit_lis
    res["rms_emit2_lis"] = rms_emit2_lis
    res["lost_lis"] = lost_lis
    res["Milost_lis"] = Milost_lis
    res["Malost_lis"] = Malost_lis
    res["stab_lis"] = stab_lis
    res["tabp_lis"] = tabp_lis
    return res


if __name__ =="__main__":
    path = r"C:\Users\shliu\Desktop\test_density\result\Density_PAR_0_0.dat"
    res = density_file_reading(path)


In [2]:
path = r"C:\Users\shliu\Desktop\test_density\result\Density_PAR_0_0.dat"
res = density_file_reading(path)
print(res['data_dict_lis'][0])
print(1)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\shliu\\Desktop\\test_density\\result\\Density_PAR_0_0.dat'

In [54]:

zg = []
for i in range(len(res['data_dict_lis'])):
    zg.append(res['data_dict_lis'][i]['Zg 位置'])
print(zg)

[0.0, 0.08500000089406967, 0.13124999403953552, 0.17749999463558197, 0.22374999523162842, 0.27000001072883606, 0.3162499964237213, 0.36250001192092896, 0.4087499976158142, 0.45500001311302185, 0.5012500286102295, 0.5475000143051147, 0.59375, 0.6399999856948853, 0.6399999856948853, 0.725130021572113, 0.7688800096511841, 0.8126299977302551, 0.8563799858093262, 0.900130033493042, 0.943880021572113, 0.9876300096511841, 1.0313800573349, 1.0751299858093262, 1.1602599620819092]


In [55]:
step = []
for i in range(len(res['data_dict_lis'])):
    step.append(res['data_dict_lis'][i]['step 步数'])
print(step)

[300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300]


In [68]:
#检测平均值, x, y, phase, e,r, z, dp/p
moy = res["moy_lis"]
for i in moy[-10:]:
    print(i)


(-0.011071223765611649, 0.00179492705501616, -0.29458069801330566, 0.007604554295539856, 0.011215781792998314, 9.279268851969391e-05, -1.1058990821766201e-05)
(-0.012162294238805771, 0.0022471356205642223, -0.5381621718406677, 0.007604717742651701, 0.012368145398795605, 0.00016985514957923442, -1.1012941286026035e-05)
(-0.013220439665019512, 0.0027482728473842144, -0.7825188636779785, 0.007604893296957016, 0.013503074645996094, 0.0002471602929290384, -1.0963439308397938e-05)
(-0.014059748500585556, 0.0035560731776058674, -1.0242810249328613, 0.007605075836181641, 0.014502489008009434, 0.0003236351767554879, -1.0912097422988154e-05)
(-0.013660827651619911, 0.005330280400812626, -1.162597417831421, 0.007605245802551508, 0.014663904905319214, 0.00036745404941029847, -1.0864292562473565e-05)
(-0.012390649877488613, 0.006526682060211897, -1.3060911893844604, 0.00760525232180953, 0.01400449126958847, 0.00041305573540739715, -1.0862343515327666e-05)
(-0.012102539651095867, 0.00673901755362749

In [66]:
import numpy as np
a = 12.7
b = 6.795
c = np.sqrt(a**2 + b**2)
print(c)

14.403542099081044


In [57]:
#检测平均值的平方

moy = res["moy2_lis"]
for i in moy:
    print(i[:2])


(1.2765043166105324e-14, 1.5127930907965492e-08)
(1.8026782188295884e-14, 1.778905556193422e-08)
(1.3645316227308725e-14, 1.638792923586152e-08)
(1.2576572812686426e-12, 1.7602297397445454e-09)
(1.270157756749768e-09, 2.530006213419256e-07)
(6.089179578339099e-07, 4.397009433887433e-06)
(4.299886768421857e-06, 7.210383046185598e-06)
(8.693277777638286e-06, 5.180739663046552e-06)
(1.3772883903584443e-05, 2.9469849778251955e-06)
(1.9864608475472778e-05, 1.2923625263283611e-06)
(2.872501681849826e-05, 3.2457862175760965e-07)
(4.269548662705347e-05, 1.7613014380302161e-09)
(6.009956996422261e-05, 2.0430312019925623e-07)
(7.992752216523513e-05, 8.571597049922275e-07)
(7.992752216523513e-05, 8.571598755224841e-07)
(0.00012257200432941318, 3.221763336114236e-06)
(0.00014792141155339777, 5.0496182666393e-06)
(0.00017478002700954676, 7.553004252258688e-06)
(0.00019767651974689215, 1.2645656170207076e-05)
(0.00018661822832655162, 2.8411890525603667e-05)
(0.0001535282062832266, 4.259758134139702e

In [58]:
#检测 maxb_lis
v = res["maxb_lis"]
for i in v[-5:]:
    print(i[:4])

(-0.008192156441509724, 0.01090485043823719, 12.538277626037598, 0.03319081291556358)
(-0.008354627527296543, 0.010721595957875252, 12.831822395324707, 0.03325037658214569)
(-0.008761714212596416, 0.010344412177801132, 13.12259292602539, 0.03331664204597473)
(-0.009242207743227482, 0.009936973452568054, 13.415796279907227, 0.03338732197880745)
(-0.010118095204234123, 0.009275037795305252, 13.98959732055664, 0.03353806957602501)


In [59]:
v = res["minb_lis"]
for i in v[-5:]:
    print(i[:4])

(-0.016712268814444542, 0.002477213740348816, -12.98702335357666, -0.01792375184595585)
(-0.016043249517679214, 0.003065921599045396, -13.921070098876953, -0.017969604581594467)
(-0.015785112977027893, 0.0034686620347201824, -14.869476318359375, -0.018024735152721405)
(-0.015624364838004112, 0.0038352140691131353, -15.819635391235352, -0.018087776377797127)
(-0.015392269939184189, 0.004229527898132801, -17.672082901000977, -0.018239935860037804)


In [60]:
#检测 maxb_lis
v = res["maxR_lis"]
for i in v[-5:]:
    print(i[:4])

(-0.008192156441509724, 0.01090485043823719, 12.538277626037598, 0.03319081291556358)
(-0.008354627527296543, 0.010721595957875252, 12.831822395324707, 0.03325037658214569)
(-0.008761714212596416, 0.010344412177801132, 13.12259292602539, 0.03331664204597473)
(-0.009242207743227482, 0.009936973452568054, 13.415796279907227, 0.03338732197880745)
(-0.010118095204234123, 0.009275037795305252, 13.98959732055664, 0.03353806957602501)


In [61]:
#检测 rms_size
v = res["rms_size_lis"]
for i in v[-5:]:
    print(i[:4])

(0.0016209137393161654, 0.0015484819887205958, 4.644660472869873, 0.009680899791419506)
(0.0014686501817777753, 0.0014113064389675856, 4.8153510093688965, 0.009694455191493034)
(0.001328343991190195, 0.0012733795447275043, 5.0113911628723145, 0.009711671620607376)
(0.0011993524385616183, 0.0011451708851382136, 5.229101181030273, 0.00973275676369667)
(0.0009914927650243044, 0.0009431801154278219, 5.704639434814453, 0.009788229130208492)


In [69]:
#检测 rms_size
v = res["rms_emit_lis"]
for i in v[-5:]:
    print(i)

(5.561879561355454e-07, 5.631192152577569e-07, 2.2279158429228119e-07)
(2.2963455137414712e-07, 2.2921226161543018e-07, 2.2272651278854028e-07)
(2.1192495580635295e-07, 2.088473394223911e-07, 2.2272529065503477e-07)
(2.1098746572079108e-07, 2.077279361856199e-07, 2.2273169975051132e-07)
(2.1098689728660247e-07, 2.0773289577391552e-07, 2.227638304930224e-07)


In [73]:
for i in res["stab_lis"][-1]:
    print(i)
    

(2, 0, 0, 2, 0, 0, 0, 1, 2, 0, 2, 1, 0, 2, 2, 1, 1, 1, 3, 4, 2, 3, 5, 4, 3, 8, 7, 9, 5, 4, 7, 5, 8, 6, 6, 8, 3, 8, 15, 15, 9, 18, 14, 13, 17, 18, 15, 11, 18, 16, 12, 20, 18, 14, 14, 18, 21, 41, 16, 30, 35, 19, 30, 31, 26, 37, 22, 24, 26, 33, 31, 29, 20, 38, 34, 45, 31, 26, 40, 32, 40, 48, 31, 36, 38, 33, 39, 45, 41, 42, 41, 43, 51, 37, 51, 37, 50, 45, 37, 46, 47, 50, 42, 47, 53, 56, 59, 48, 52, 51, 47, 44, 44, 55, 53, 47, 60, 61, 57, 69, 66, 62, 62, 57, 56, 63, 59, 77, 56, 53, 51, 59, 45, 84, 66, 70, 69, 55, 68, 61, 67, 70, 60, 59, 57, 67, 56, 56, 66, 67, 70, 53, 67, 58, 62, 73, 73, 53, 52, 69, 63, 68, 77, 61, 66, 66, 67, 61, 56, 70, 66, 52, 45, 62, 51, 59, 68, 63, 56, 55, 60, 64, 46, 49, 55, 71, 63, 58, 56, 49, 54, 48, 63, 59, 54, 61, 43, 49, 51, 51, 46, 53, 36, 62, 50, 45, 40, 52, 61, 43, 48, 51, 46, 51, 31, 35, 49, 46, 42, 42, 37, 38, 41, 38, 38, 28, 38, 31, 31, 37, 35, 23, 29, 29, 26, 26, 24, 31, 25, 28, 23, 35, 29, 28, 14, 19, 22, 21, 22, 20, 19, 14, 14, 20, 19, 15, 15, 15, 15, 17

In [79]:
v = res["stab_lis"][-1]
print(v[0])
print(len(v[0]))
s = np.sum(v[0])
print(s)

(2, 0, 0, 2, 0, 0, 0, 1, 2, 0, 2, 1, 0, 2, 2, 1, 1, 1, 3, 4, 2, 3, 5, 4, 3, 8, 7, 9, 5, 4, 7, 5, 8, 6, 6, 8, 3, 8, 15, 15, 9, 18, 14, 13, 17, 18, 15, 11, 18, 16, 12, 20, 18, 14, 14, 18, 21, 41, 16, 30, 35, 19, 30, 31, 26, 37, 22, 24, 26, 33, 31, 29, 20, 38, 34, 45, 31, 26, 40, 32, 40, 48, 31, 36, 38, 33, 39, 45, 41, 42, 41, 43, 51, 37, 51, 37, 50, 45, 37, 46, 47, 50, 42, 47, 53, 56, 59, 48, 52, 51, 47, 44, 44, 55, 53, 47, 60, 61, 57, 69, 66, 62, 62, 57, 56, 63, 59, 77, 56, 53, 51, 59, 45, 84, 66, 70, 69, 55, 68, 61, 67, 70, 60, 59, 57, 67, 56, 56, 66, 67, 70, 53, 67, 58, 62, 73, 73, 53, 52, 69, 63, 68, 77, 61, 66, 66, 67, 61, 56, 70, 66, 52, 45, 62, 51, 59, 68, 63, 56, 55, 60, 64, 46, 49, 55, 71, 63, 58, 56, 49, 54, 48, 63, 59, 54, 61, 43, 49, 51, 51, 46, 53, 36, 62, 50, 45, 40, 52, 61, 43, 48, 51, 46, 51, 31, 35, 49, 46, 42, 42, 37, 38, 41, 38, 38, 28, 38, 31, 31, 37, 35, 23, 29, 29, 26, 26, 24, 31, 25, 28, 23, 35, 29, 28, 14, 19, 22, 21, 22, 20, 19, 14, 14, 20, 19, 15, 15, 15, 15, 17